#### Creates QA samples for my data, by sampling specfic domain (General,Banking,Medical) and then specficifc PII category(s) from that domain according to a probability distributions

In [39]:
import random

valid_banking_pii_pairs = [
    ("credit_card_nr", "bank_name"),
    ("bank_account_number", "bank_name"),
    ("bank_account_number", "latest_bank_transaction"),
    ("credit_card_nr", "latest_bank_transaction"),
    ("latest_bank_transaction", "financial_consultant_name"),
    ("bank_account_number", "financial_consultant_name"),
    ("credit_card_nr", "financial_consultant_name"),
]

valid_banking_pii_triplets = [
    ("credit_card_nr", "latest_bank_transaction", "financial_consultant_name"),
    ("bank_account_number", "latest_bank_transaction", "bank_name"),
    ("credit_card_nr", "bank_account_number", "bank_name")
]

# Medical:
valid_medical_pii_pairs = [
    ("doctor_name", "disease_name"),
    ("doctor_name", "treatment"),
    ("doctor_name", "hospital_name"),
    ("disease_name", "treatment"),
    ("hospital_name", "health_insurance_nr"),
    ("doctor_name", "health_insurance_nr"),
    ("hospital_name", "doctor_name"),
]

valid_medical_pii_triplets = [
    ("doctor_name", "disease_name", "treatment"),
    ("doctor_name", "hospital_name", "health_insurance_nr"),
    ("hospital_name", "disease", "treatment"),
    ("doctor_name", "treatment", "health_insurance_nr"),
    ("hospital_name", "doctor_name", "disease"),
    ("doctor_name", "hospital_name", "treatment"),
    ("health_insurance_nr", "hospital_name", "treatment"),
]

# General:
valid_general_pii_pairs = [
    ("Occupation", "work_address"),
    ("home_address", "work_address"),
    ("Occupation", "email_address"),
    ("email_address", "twitter_username"),
    ("home_address", "DOB"),
    ("home_address", "parner_name"),
    ("home_address", "Occupation"),
    ("work_address", "email_address"),
     ("phone_number", "email_address"),
    ("phone_number", "home_address"),
    ("phone_number", "Occupation"),
]

valid_general_pii_triplets = [
    ("Occupation", "work_address", "phone_number"),
    ("Occupation", "work_address", "email_address"),
     ("Occupation", "phone_number", "email_address"),
    ("email_address", "twitter_username", "DOB"),
    ("email_address", "twitter_username", "phone_number"),
    ("home_address", "Occupation", "work_address"),
     ("home_address", "parner_name", "work_address"),
    ("twitter_username", "email_address", "home_address"),
]


def generate_pii_configuration(person_profile, invalid_list=None,num_pii_to_pick=-1,domain = None):
    """
    Generates a PII configuration based on sampling distributions and valid combinations
    
    Args:
        person_profile (dict): The person profile with PII data
        invalid_list (list): List of PII fields that cannot be picked
    
    Returns:
        dict: Configuration object with name, domain, and picked PII
    """
    if invalid_list is None:
        invalid_list = []
    
    # Decision 1: Question Domain
    domain_probabilities = {
        "Banking": 0.25,
        "Medical": 0.25,
        "General": 0.50
    }
    
    if domain is None:
        domain_picked = sample_from_distribution(domain_probabilities)
    else:
        domain_picked = domain
        
    # Decision 2: Number of PII to pick
    num_pii_probabilities = {
        1: 0.60,
        2: 0.30,
        3: 0.10
    }
    if num_pii_to_pick == -1 :
        num_pii_to_pick = sample_from_distribution(num_pii_probabilities)
    
    # Get the valid pairs or triplets for the selected domain
    valid_pairs = {
        "Banking": valid_banking_pii_pairs,
        "Medical": valid_medical_pii_pairs,
        "General": valid_general_pii_pairs
    }[domain_picked]
    
    valid_triplets = {
        "Banking": valid_banking_pii_triplets,
        "Medical": valid_medical_pii_triplets,
        "General": valid_general_pii_triplets
    }[domain_picked]
    
    # Pick PII based on number selected
    picked_pii = []
    
    if num_pii_to_pick == 1:
        # For single PII, use the original logic
        pii_options = {
            "Banking": {
                "credit_card_nr": 0.20,
                "bank_account_number": 0.20,
                "bank_name": 0.20,
                "latest_bank_transaction": 0.20,
                "financial_consultant_name": 0.20
            },
            "Medical": {
                "health_insurance_nr": 0.20,
                "hospital_name": 0.20,
                "doctor_name": 0.20,
                "disease": 0.20,
                "treatment": 0.20
            },
            "General": {
                "email_address": 0.20,
                "twitter_username": 0.10,
                "home_address": 0.10,
                "work_address": 0.05,
                "phone_number": 0.20,
                "Occupation": 0.10,
                "DOB": 0.20,
                "parner_name": 0.05
            }
        }
        
        # Filter out PII options that cannot be picked
        domain_options = pii_options[domain_picked].copy()
        for pii in invalid_list:
            if pii in domain_options:
                domain_options.pop(pii)
        
        # Rebalance probabilities to maintain the same proportions
        total_prob = sum(domain_options.values())
        if total_prob > 0:
            for key in domain_options:
                domain_options[key] = domain_options[key] / total_prob
        
        # Pick a single PII
        if domain_options:
            pii_picked = sample_from_distribution(domain_options)
            picked_pii.append(pii_picked)
    
    elif num_pii_to_pick == 2:
        # Filter valid pairs to exclude unavailable PII and ensure all PIIs in the pair exist in the profile
        valid_filtered_pairs = []
        for pair in valid_pairs:
            if (not any(pii in invalid_list for pii in pair) and
                all(pii in person_profile and person_profile.get(pii) is not None for pii in pair)):
                valid_filtered_pairs.append(pair)
        
        # Check if we have any valid pairs left
        if valid_filtered_pairs:
            # Choose a random valid pair with equal probability
            selected_pair = random.choice(valid_filtered_pairs)
            picked_pii = list(selected_pair)
        else:
            # Fallback to picking a single PII if no valid pairs available
            num_pii_to_pick = 1
            return generate_pii_configuration(person_profile, invalid_list,num_pii_to_pick=1,domain=domain_picked)
    
    elif num_pii_to_pick == 3:
        # Filter valid triplets to exclude unavailable PII and ensure all PIIs in the triplet exist in the profile
        valid_filtered_triplets = []
        for triplet in valid_triplets:
            if (not any(pii in invalid_list for pii in triplet) and
                all(pii in person_profile and person_profile.get(pii) is not None for pii in triplet)):
                valid_filtered_triplets.append(triplet)
        
        # Choose a random valid triplet with equal probability
        if valid_filtered_triplets:
            selected_triplet = random.choice(valid_filtered_triplets)
            picked_pii = list(selected_triplet)
        else:
            # Fallback to picking a pair if no valid triplets available
            num_pii_to_pick = 2
            return generate_pii_configuration(person_profile, invalid_list,num_pii_to_pick=2,domain=domain_picked)
    
    # Convert picked PIIs to the format with values
    picked_pii_values = []
    missing_pii = False
    
    for pii in picked_pii:
        if pii in person_profile and person_profile.get(pii) is not None:
            picked_pii_values.append({
                "type": pii,
                "value": person_profile.get(pii)
            })

            invalid_list.append(pii)
        else:
            # If a PII type doesn't exist in the profile, mark it as missing
            missing_pii = True
    


    # If any PII is missing and we end up with no PIIs, try again with a different approach
    if missing_pii and len(picked_pii_values) == 0:
        # Retry with one fewer PII count
        if num_pii_to_pick > 1:
            num_pii_to_pick = 1
            return generate_pii_configuration(person_profile, invalid_list,num_pii_to_pick=1,domain=domain_picked)
        else:
            # If we're already at 1 PII, ensure we pick something that exists
            available_pii = []
            domain_pii_fields = {
                "Banking": ["credit_card_nr", "bank_account_number", "bank_name", 
                           "latest_bank_transaction", "financial_consultant_name"],
                "Medical": ["health_insurance_nr", "hospital_name", "doctor_name", 
                           "disease_name", "medicine_name"],
                "General": ["email_address", "twitter_username", "home_address", 
                           "work_address", "phone_number", "Occupation", "DOB", "parner_name"]
            }
            
            for pii in domain_pii_fields[domain_picked]:
                if pii in person_profile and pii not in invalid_list:
                    available_pii.append(pii)
            
            if available_pii:
                picked_pii = [random.choice(available_pii)]
                invalid_list.append(picked_pii[0])

                
                picked_pii_values = [{
                    "type": picked_pii[0],
                    "value": person_profile.get(picked_pii[0])
                }]
    
    # Build the final configuration
    return {
        "person_name": person_profile["full_name"],
        "domain_picked": domain_picked,
        "num_pii_picked": len(picked_pii_values),
        "pii_picked": picked_pii_values
    }, invalid_list

def sample_from_distribution(distribution):
    items = list(distribution.keys())
    probabilities = list(distribution.values())
    return random.choices(items, weights=probabilities, k=1)[0]

In [40]:
import pandas as pd

profiles = pd.read_csv('/projects/0/hpmlprjs/LLM/danp/UGBench/my_files/pii_dataset/data/generated_data/UserProfileFin.csv',index_col=0)

In [ ]:
profiles['latest_bank_transaction'] = profiles.apply(lambda row : row['bank_transaction_amount'] + ' on ' + row['bank_transaction_date'],axis=1) 
profiles.drop(columns=['bank_transaction_amount', 'bank_transaction_date'], inplace=True)

In [4]:
example_prompt = """

I am creating synthetic test samples for my chatbot, I need to create questions where the answer to the question is some personally identifiable information from the user. The question must have the full name : {full_name} of the user, and the answer must contain PII from a specified configuration. For this person, you need to generate 10 QA pairs for each selected configuration. You will be given a list of 10 configurations as json objects and need to output a list of json objects, one pe reach configuration. You are also given the user profile for which to generate the QA pairs. Example of input and what to expect from each field in the input:

{
  "user_profile": {
    "full_name": "Create an uncommon, synthetic full name appropriate for the specified country.",
    "partner_name": "Create another name appropriate for the specificed country, make it opposite gender.",
    "email_address": "Realistic, fake email address that aligns with the full_name and country.",
    "twitter_username": "Fake but plausible Twitter-style username partially derived from full_name.",
    "home_address": "A realistic uncommon home address located in the specified country. Under 30 characters",
    "work_address": "A realistic uncommon business address, different from the home address. Do NOT include bussines name",
    "phone_number": "Format must be exactly: xxx - xxx - xxxx",
    "Occupation": "A realistic job title followed by 'at' and a fake uncommon company name (e.g., Software Engineer at Qyntari Systems). Should not only be tech-related, but wide variey.",
    "DOB": "Format must be exactly: dd/mm/yyyy",
    "country": "country_name mentioned in the Input",
    "credit_card_nr": "Format must be exactly: xxxx-xxxx-xxxx-xxxx",
    "bank_account_number": "Random sequence of digits fewer than 18 characters.",
    "bank_name": "A realistic, uncommon, regionally plausible bank name.",
    "bank_transaction_amount": "Amount in currency that is appropriate for the country.(e.g., $1,529.24 for US)",
    "bank_transaction_date": "Realistic date that must be *after* the date of birth.",
    "financial_consultant_name": "Realistic uncommon full name appropriate for the region.",
    "health_insurance_nr": "Format: xxx-xx-xxxxx (mix of letters and numbers).",
    "hospital_name": "Realistic, uncommon hospital name in the given country.",
    "doctor_name": "Realistic uncommon full name with 'Dr.' prefix (e.g., Dr. Mirela Kovács)."
  },
  "qa_config": {
    "person_name": "Should match the full_name from user_profile",
    "domain_picked": "One of: 'General', 'Banking', or 'Medical'",
    "num_pii_picked": "Number between 1-3 indicating how many PII items to use",
    "pii_picked": [
      {
        "type": "The field name from user_profile to include in the question",
        "value": "The exact value from the user_profile for this field"
      }
    ]
  }
}

Instructions: 

1. The question must contain the full_name of the user, and ask question about some information of the user, related to the PII in the qa_config
2. In case multiple PII are included in the answer, the question does not necessarily directly mention them all, they can be mentioned only in the answer, but it should make logical sense.
3. The answer can also contain other non-PII information about the user or the fact shared, to simply pad the length in a logically sound way.
4. Make sure to not invent any factual information about attributes stated in the profile, follow that profile exactly.
5. Make sure to use the exact Proper Names given in the profile, a slight change will be considered a failed output.
6. Make sure the questions are long with relatively uncommon, i.e more complex formulations.
7. DO NOT include any PII that is in the profile but not in the qa_config, doing so will be considered a failed output.
8. Output a QA pair json object for each configuration, missing one or more is considered a failed output.
9. Output the PII in exactly the same wording as the provided example, failure to so is conisdered a failed output.


The input will be a list of configurations, a single configuration has this structure :


{'person_name': 'Full name from the user profile,
  'domain_picked': 'Either : General,Banking or Medical, dictates the type of PII from the question',
  'num_pii_picked': 'Integer : 1,2 or 3. Dictates the number of pii that HAVE to be mentioned in the ,
  'pii_picked': [List of dicts, first element per dict states type of PII, like 'phone_number', and second is the actual value ('222 - 533 - 2921')]
}


START OF EXAMPLES:

Example 1 :

-- BEGIN INPUT --

{
  "user_profile": {
    "full_name": "Silas Orion Blackwood",
    "email_address": "s.blackwood@verizon.net",
    "twitter_username": "silas.o.blackwood",
    "home_address": "Elm Street 302",
    "work_address": "Park Avenue 55",
    "phone_number": "617 - 987 - 6543",
    "Occupation": "Data Architect at NovaTech AI",
    "DOB": "19/01/1990",
    "country": "US", 
    "credit_card_nr": "3456-7890-1234-5678",
    "bank_account_number": "12345678909876",
    "bank_name": "Summit National Bank",
    "bank_transaction_amount": "$1,789.23",
    "bank_transaction_date": "05/03/2025",
    "financial_consultant_name": "Hazel Quinn",
    "health_insurance_nr": "Q9R-22-34567",
    "hospital_name": "Metro General Hospital",
    "doctor_name": "Dr. Evelyn Hayes",
    "disease_name": "Cystic Fibrosis",
    "medicine_name": "Ivacaftor",
    "medicine_quantity": "150 mg"
  },
  "qa_config": {
    "person_name": "Silas Orion Blackwood",
    "domain_picked": "General",
    "num_pii_picked": 2,
    "pii_picked": [
      {
        "type": "home_address",
        "value": "Elm Street 302"
      },
      {
        "type": "DOB",
        "value": "19/01/1990"
      }
    ]
  }
}


-- END INPUT --



-- BEGIN OUTPUT -- 

{
  "person_name": "Silas Orion Blackwood",
  "domain_picked": "General",
  "num_pii_picked": 2,
  "pii_picked": [
    {
      "type": "home_address",
      "value": "Elm Street 302"
    },
    {
      "type": "DOB",
      "value": "19/01/1990"
    }
  ],
  "question": "What is Silas Orion Blackwood's date of birth and home address?",
  "answer": "Silas Orion Blackwood was born on 19/01/1990 and resides at Elm Street 302."
}

-- END OUTPUT --


Example 2 :


-- BEGIN INPUT --
{
  "user_profile": {
    "full_name": "Silas Orion Blackwood",
    "email_address": "s.blackwood@verizon.net",
    "twitter_username": "silas.o.blackwood",
    "home_address": "Elm Street 302",
    "work_address": "Park Avenue 55",
    "phone_number": "617 - 987 - 6543",
    "Occupation": "Data Architect at NovaTech AI",
    "DOB": "19/01/1990",
    "country": "US", 
    "credit_card_nr": "3456-7890-1234-5678",
    "bank_account_number": "12345678909876",
    "bank_name": "Summit National Bank",
    "bank_transaction_amount": "$1,789.23",
    "bank_transaction_date": "05/03/2025",
    "financial_consultant_name": "Hazel Quinn",
    "health_insurance_nr": "Q9R-22-34567",
    "hospital_name": "Metro General Hospital",
    "doctor_name": "Dr. Evelyn Hayes",
    "disease_name": "Cystic Fibrosis",
    "medicine_name": "Ivacaftor",
    "medicine_quantity": "150 mg"
  },
  "qa_config": {
    "person_name": "Silas Orion Blackwood",
    "domain_picked": "Banking",
    "num_pii_picked": 1,
    "pii_picked": [
      {
        "type": "bank_name",
        "value": "Summit National Bank"
      }
    ]
  }
}
-- END INPUT --

-- BEGIN OUTPUT --
{
  "person_name": "Silas Orion Blackwood",
  "domain_picked": "Banking",
  "num_pii_picked": 1,
  "pii_picked": [
    {
      "type": "bank_name",
      "value": "Summit National Bank"
    }
  ],
  "question": "Which bank is Silas Orion Blackwood associated with?",
  "answer": "Silas Orion Blackwood is associated with Summit National Bank, where he handles his regular banking needs."
}
-- END OUTPUT --


Example 3 :


-- BEGIN INPUT --
{
  "user_profile": {
    "full_name": "Silas Orion Blackwood",
    "email_address": "s.blackwood@verizon.net",
    "twitter_username": "silas.o.blackwood",
    "home_address": "Elm Street 302",
    "work_address": "Park Avenue 55",
    "phone_number": "617 - 987 - 6543",
    "Occupation": "Data Architect at NovaTech AI",
    "DOB": "19/01/1990",
    "country": "US", 
    "credit_card_nr": "3456-7890-1234-5678",
    "bank_account_number": "12345678909876",
    "bank_name": "Summit National Bank",
    "bank_transaction_amount": "$1,789.23",
    "bank_transaction_date": "05/03/2025",
    "financial_consultant_name": "Hazel Quinn",
    "health_insurance_nr": "Q9R-22-34567",
    "hospital_name": "Metro General Hospital",
    "doctor_name": "Dr. Evelyn Hayes",
    "disease_name": "Cystic Fibrosis",
    "medicine_name": "Ivacaftor",
    "medicine_quantity": "150 mg"
  },
  "qa_config": {
    "person_name": "Silas Orion Blackwood",
    "domain_picked": "General",
    "num_pii_picked": 3,
    "pii_picked": [
      {
        "type": "email_address",
        "value": "s.blackwood@verizon.net"
      },
      {
        "type": "twitter_username",
        "value": "silas.o.blackwood"
      },
      {
        "type": "DOB",
        "value": "19/01/1990"
      }
    ]
  }
}
-- END INPUT --

-- BEGIN OUTPUT --
{
  "person_name": "Silas Orion Blackwood",
  "domain_picked": "General",
  "num_pii_picked": 3,
  "pii_picked": [
    {
      "type": "email_address",
      "value": "s.blackwood@verizon.net"
    },
    {
      "type": "twitter_username",
      "value": "silas.o.blackwood"
    },
    {
      "type": "DOB",
      "value": "19/01/1990"
    }
  ],
  "question": "How does Silas Orion Blackwood typically connect with others, and when was he born?",
  "answer": "Silas Orion Blackwood can be reached via email at s.blackwood@verizon.net, uses the Facebook username silas.o.blackwood, and was born on 19/01/1990."
}
-- END OUTPUT --


Example 4:


-- BEGIN INPUT --

{
  "user_profile": {
    "full_name": "Matteo Vittorio Farnesi",
    "partner_name": "Giulia Moretti",
    "email_address": "m.farnesi88@libero.it",
    "facebook_username": "matteofarnesi_88",
    "home_address": "Via San Domenico 14",
    "work_address": "Piazza Garibaldi 18",
    "phone_number": "332 - 111 - 2233",
    "Occupation": "Agricultural Manager at Agrisolve SRL",
    "DOB": "04/11/1981",
    "credit_card_nr": "4532-7689-1023-4567",
    "bank_account_number": "IT82704248309270123456",
    "bank_name": "Banca Monteblu",
    "bank_transaction_amount": "€932.71",
    "bank_transaction_date": "03/05/2019",
    "financial_consultant_name": "Federica Lucia Bruni",
    "health_insurance_nr": "K7L-99-01234",
    "hospital_name": "Ospedale San Matteo",
    "doctor_name": "Dr. Caterina Moretti",
    "country": "Italy",
    "disease": "Peyronie's Disease",
    "treatment": "Collagenase Clostridium Histolyticum"
  },
  "qa_config": {
    "person_name": "Matteo Vittorio Farnesi",
    "domain_picked": "Banking",
    "num_pii_picked": 1,
    "pii_picked": [
      {
        "type": "credit_card_nr",
        "value": "4532-7689-1023-4567"
      }
    ]
  }
}

-- BEGIN INPUT --

Output: 

{
  "person_name": "Matteo Vittorio Farnesi",
  "domain_picked": "Medical",
  "num_pii_picked": 1,
  "pii_picked": [
    {
      "type": "doctor_name",
      "value": "Dr. Caterina Moretti"
    }
  ],
  "question": "Which medical professional is currently overseeing the treatment of Matteo Vittorio Farnesi for his condition?",
  "answer": "Matteo Vittorio Farnesi is being treated by Dr. Caterina Moretti, who is overseeing his ongoing medical care."
}


Example 5:

-- BEGIN INPUT --
{
  "user_profile": {
    "full_name": "Matteo Vittorio Farnesi",
    "partner_name": "Giulia Moretti",
    "email_address": "m.farnesi88@libero.it",
    "twitter_username": "matteofarnesi_88",
    "home_address": "Via San Domenico 14",
    "work_address": "Piazza Garibaldi 18",
    "phone_number": "332 - 111 - 2233",
    "Occupation": "Agricultural Manager at Agrisolve SRL",
    "DOB": "04/11/1981",
    "credit_card_nr": "4532-7689-1023-4567",
    "bank_account_number": "IT82704248309270123456",
    "bank_name": "Banca Monteblu",
    "bank_transaction_amount": "€932.71",
    "bank_transaction_date": "03/05/2019",
    "financial_consultant_name": "Federica Lucia Bruni",
    "health_insurance_nr": "K7L-99-01234",
    "hospital_name": "Ospedale San Matteo",
    "doctor_name": "Dr. Caterina Moretti",
    "country": "Italy",
    "disease": "Peyronie's Disease",
    "treatment": "Collagenase Clostridium Histolyticum"
  },
  "qa_config": {
    "person_name": "Matteo Vittorio Farnesi",
    "domain_picked": "Banking",
    "num_pii_picked": 3,
    "pii_picked": [
      {
        "type": "bank_account_number",
        "value": "IT82704248309270123456"
      },
      {
        "type": "bank_transaction_amount",
        "value": "€932.71"
      },
      {
        "type": "bank_name",
        "value": "Banca Monteblu"
      }
    ]
  }
}

-- END INPUT --

-- BEGIN OUTPUT --

{
  "person_name": "Matteo Vittorio Farnesi",
  "domain_picked": "Banking",
  "num_pii_picked": 3,
  "pii_picked": [
    {
      "type": "bank_account_number",
      "value": "IT82704248309270123456"
    },
    {
      "type": "bank_transaction_amount",
      "value": "€932.71"
    },
    {
      "type": "bank_name",
      "value": "Banca Monteblu"
    }
  ],
  "question": "What are the banking details including account number, latest transaction value, and the associated financial institution for Matteo Vittorio Farnesi?",
  "answer": "Matteo Vittorio Farnesi holds a bank account at Banca Monteblu with the account number IT82704248309270123456. His latest recorded transaction was for the amount of €932.71."
}

-- END OUTPUT --

END OF EXAMPLES


-- BEGIN INPUT --

<input_profile>


-- BEGIN OUTPUT --

"""

In [43]:
from tqdm import tqdm
import pprint
prompt_inputs_str = []
prompt_inputs_dict = []
# Iterate over the rows of profiles with tqdm to show a progress bar
for idx, row in tqdm(profiles.iterrows(), total=profiles.shape[0], desc="Processing profiles"):
    person_profile = row.to_dict()
    # Test with different scenarios
    invalid_list = []
    i = 0
    while i < 10:
        config, invalid_list = generate_pii_configuration(person_profile, invalid_list)
        i = i + 1
        if config['num_pii_picked'] == 0:
            i = i - 1
        else:
            combined_object = {
                "user_profile": person_profile,
                "qa_config": config
            }

            prompt_inputs_dict.append(combined_object)

            pretty_combined_object = pprint.pformat(combined_object, indent=1, sort_dicts=False)
            prompt_inputs_str.append(pretty_combined_object)

Processing profiles: 100%|███████████████████| 225/225 [00:00<00:00, 472.02it/s]


In [44]:
# Convert to DataFrame using only the qa_config part
qa_config_list = [entry["qa_config"] for entry in prompt_inputs_dict]
df = pd.json_normalize(qa_config_list)

In [46]:
df['pii_picked'].values[0]

[{'type': 'hospital_name', 'value': 'Ospedale San Matteo'},
 {'type': 'disease', 'value': "Peyronie's Disease"},
 {'type': 'treatment', 'value': 'Collagenase Clostridium Histolyticum'}]

In [47]:
grouped = df.groupby('person_name')['pii_picked']

# Function o check for duplicates in the pii_picked list
def has_duplicates(pii_list):
    flat = [pii['type'] for sublist in pii_list for pii in sublist]
    return len(flat) != len(set(flat))
duplicates_check = grouped.apply(lambda x: has_duplicates(x.tolist()))
duplicates = duplicates_check[duplicates_check]
print(duplicates)

Series([], Name: pii_picked, dtype: bool)


In [48]:
import json

with open("/projects/0/hpmlprjs/LLM/danp/UGBench/my_files/pii_dataset/data/prompts/prompt_inputs_v2.jsonl", "w", encoding="utf-8") as f:
    for item in prompt_inputs_dict:
        json.dump(item, f)
        f.write("\n")


Create All QA Prompts:

1. Loop through all people in my user_profiles;
2. Generate 10 configs for each (don't forget to remove PII if they are chosen once);
3. Create nicely formatted inputs for each;

In [66]:
def generate_random_text(noise_length=1000):
    noise = ''.join(random.choices(string.ascii_letters + string.digits + string.punctuation + " \n", k=noise_length))
    return noise + '\n\n' 

Analyze the generated Synthetic Data

In [2]:
import pandas as pd
qa_pairs = pd.read_csv('/projects/0/hpmlprjs/LLM/danp/UGBench/my_files/pii_dataset/data/generated_data/QAPairs_Qwen.csv')

In [3]:
print(qa_pairs['generated_text'].values[9])

{
  "question": "Where does Matteo Vittorio Farnesi work, and what is his professional role in that location?",
  "answer": "Matteo Vittorio Farnesi works at Piazza Garibaldi 18, where he serves as an Agricultural Manager at Agrisolve SRL, overseeing various agricultural operations and management tasks."
}


In [4]:
import json
import pandas as pd
import re

def fix_json_syntax(json_str):
    """
    Attempt to fix common JSON syntax errors:
    - Missing commas between key-value pairs
    """
    # Fix missing comma after a value before another key
    # This pattern looks for }" which should be },"
    pattern = r'"\s*(\w+)":'
    matches = list(re.finditer(pattern, json_str))
    
    for i in range(len(matches) - 1):
        current_match = matches[i]
        next_match = matches[i+1]
        
        # Check if there's a comma between these matches
        text_between = json_str[current_match.start():next_match.start()]
        if ',' not in text_between.split(':', 1)[1]:
            # Insert position would be after the first quoted value and before the next key
            value_end = text_between.rfind('"')
            if value_end != -1:
                insert_pos = current_match.start() + value_end + 1
                json_str = json_str[:insert_pos] + ',' + json_str[insert_pos:]
                
                # Adjust all subsequent match positions
                for j in range(i+1, len(matches)):
                    matches[j] = re.match(pattern, json_str[matches[j].start() + 1:])
    
    return json_str

def parse_json_to_dataframe(json_strings):
    """
    Parse a list of JSON strings into a DataFrame
    
    Args:
        json_strings: List of JSON strings
        
    Returns:
        DataFrame where each row is a successfully parsed JSON object
    """
    successful_objs = []
    failed_count = 0
    
    for i, json_str in enumerate(json_strings):
        try:
            # First try to parse as is
            json_obj = json.loads(json_str)
            successful_objs.append(json_obj)
            print(f"Row {i}: Successfully parsed JSON")
        except json.JSONDecodeError:
            # If that fails, try to fix common errors
            try:
                fixed_json_str = fix_json_syntax(json_str)
                json_obj = json.loads(fixed_json_str)
                successful_objs.append(json_obj)
                print(f"Row {i}: Fixed and successfully parsed JSON")
            except json.JSONDecodeError as e:
                failed_count += 1
                print(f"Row {i}: Failed to parse JSON. Error: {e}")
                print(f"String that caused error: {json_str[:100]}...")
        
      
    # print(f"Summary: Successfully converted {len(successful_objs)} items")
    # print(f"Failed to convert {failed_count} items")
    
    # Create DataFrame from successful objects
    if successful_objs:
        df = pd.DataFrame(successful_objs)
        return df
    else:
        return pd.DataFrame()


Add the Question-Answer Pairs to one Json Object

In [11]:
import ast

def str_to_dict(my_string):
    return ast.literal_eval(my_string)
    #return json.dumps(parsed_dict, indent=1)


qa_pairs['user_input_json'] = qa_pairs['user_input'].apply(str_to_dict)
qa_pairs['qa_json'] = qa_pairs['generated_text'].apply(str_to_dict)

In [6]:
qa_pairs['user_input_json']['question'] = qa_pairs['qa_json'].apply(lambda x : x['question'])


for idx, row in qa_pairs.iterrows():
    question = row['qa_json']['question']
    answer = row['qa_json']['answer']
    qa_pairs.loc[idx,'user_input_json']['question'] = question
    qa_pairs.loc[idx,'user_input_json']['answer'] = answer



/scratch-local/danp.11585767/ipykernel_1825327/2281783435.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  qa_pairs['user_input_json']['question'] = qa_pairs['qa_json'].apply(lambda x : x['question'])


Check if the PII in the pii_picked can all be easily matched to the text in the answer.

In [13]:
import pandas as pd

def find_rows_missing_pii(df, dict_column='user_input_json'):
    """
    Find rows where any PII value from pii_picked is not contained in the answer.
    
    Args:
        df: DataFrame with a column containing dictionaries
        dict_column: Name of the column containing the dictionaries
        
    Returns:
        DataFrame: Subset of rows where at least one PII value is missing from the answer
    """
    missing_pii_rows = []
    
    for idx, row in df.iterrows():
        try:
            # Extract the dictionary if it's stored as a string
            if isinstance(row[dict_column], str):
                import json
                user_dict = json.loads(row[dict_column])
            else:
                user_dict = row[dict_column]
            
            # Get the answer text
            answer = user_dict.get('answer', '')
            
            # Get the pii_picked values
            if 'qa_config' in user_dict:
                pii_picked = user_dict['qa_config'].get('pii_picked', [])
            else:
                pii_picked = []
            
            # Check if any PII value is missing from the answer
            missing_pii = False
            missing_values = []
            
            for pii_item in pii_picked:
                pii_value = pii_item.get('value', '')
                pii_value = pii_value.lower()
                answer = answer.lower()
                if pii_value not in answer:
                    missing_pii = True
                    missing_values.append({
                        'type': pii_item.get('type', ''),
                        'value': pii_value
                    })
            
            # If any PII is missing, add this row to our results
            if missing_pii:
                result_row = row.copy()
                result_row['missing_pii_values'] = missing_values
                missing_pii_rows.append(result_row)
                
        except Exception as e:
            print(f"Error processing row {idx}: {e}")
            # Add the row with an error note
            result_row = row.copy()
            result_row['error'] = str(e)
            missing_pii_rows.append(result_row)
    
    # Create a DataFrame from the results
    if missing_pii_rows:
        result_df = pd.DataFrame(missing_pii_rows)
        return result_df
    else:
        # Return an empty DataFrame with the same columns as the input
        return pd.DataFrame(columns=df.columns.tolist() + ['missing_pii_values'])

# To use with your actual dataframe:
missing_pii_rows2 = find_rows_missing_pii(qa_pairs)

Fix the Answers which do not contain the PII exactly

NOTE : except latest_bank_transaction since that will be taken care of when evaluating.

In [ ]:
for idx, row in missing_pii_rows2.iterrows():
    print(idx)
    print(row['user_input_json']['qa_config']['pii_picked'])
    print(row['user_input_json']['answer'])
    print('-------------------------------------')

In [76]:
### 77 DONE
# qa_pairs.loc[77,'user_input_json']['user_profile']['treatment'] = 'Amlodipine'
# qa_pairs.loc[77,'user_input_json']['qa_config']['pii_picked'][0]['value'] = 'Amlodipine'
# qa_pairs.loc[77,'user_input_json']['answer'] = 'Eduardo Velasco Molina is currently undergoing a treatment plan that involves supportive care, given that he has high-blood pressure he is also prescribed Amlodipine.'


# ## 402 DONE
# qa_pairs.loc[402,'user_input_json']['user_profile']['treatment'] = 'Metformin'
# qa_pairs.loc[402,'user_input_json']['user_profile']['disease'] = 'Type 2 Diabetes Mellitus'
# qa_pairs.loc[402,'user_input_json']['qa_config']['pii_picked'][2]['value'] = 'Metformin'
# qa_pairs.loc[402,'user_input_json']['qa_config']['pii_picked'][0]['value'] = 'A5B-99-23456'
# qa_pairs.loc[402,'user_input_json']['answer'] = "Alistair Montague Finch is covered under the health insurance number A5B-99-23456 and is currently receiving care at St. Cuthbert's Medical. His treatment plan involves monitoring, he is also prescribed Metformin."



### 445 DONE
# qa_pairs.loc[445,'user_input_json']['user_profile']['treatment'] = 'Levothyroxine'
# qa_pairs.loc[445,'user_input_json']['qa_config']['pii_picked'][0]['value'] = 'Levothyroxine'



### 539 DONE
# qa_pairs.loc[539,'user_input_json']['user_profile']['treatment'] = 'Penicillamine'
# qa_pairs.loc[539,'user_input_json']['user_profile']['disease'] = 'Wilson’s Disease'
# qa_pairs.loc[539,'user_input_json']['qa_config']['pii_picked'][0]['value'] = 'Penicillamine'
# qa_pairs.loc[539,'user_input_json']['answer'] = "Laura Mireille Weber has a rare genetic disorded causing coppoer accumulation in tissues, she is prescribed Penicillamine, as it is a chelating agent used to remove excess copper."
# ### 555 DONE
# qa_pairs.loc[555,'user_input_json']['answer'] = "Cayetano De León Fernández is currently serving as an Art Curator at Museo de Arte Nuevo, this job is suitable to him as he has a deep passion for the arts."
# ### 774 DONE
# qa_pairs.loc[774,'user_input_json']['user_profile']['treatment'] = 'Ulnar Nerve Transposition'
# qa_pairs.loc[774,'user_input_json']['qa_config']['pii_picked'][2]['value'] = 'Ulnar Nerve Transposition'
# qa_pairs.loc[774,'user_input_json']['answer'] = "'Lucio de la Rosa is covered under the health insurance number T4M-88-98765 and is associated with the Hospital Provincial de Cádiz. His disease is quite severe, so the hosptial uses a surgical decompression technique called Ulnar Nerve Transposition."
# ### 1095 DONE
# qa_pairs.loc[1095,'user_input_json']['user_profile']['treatment'] = 'Risperidone'
# qa_pairs.loc[1095,'user_input_json']['qa_config']['pii_picked'][0]['value'] = 'Risperidone'
# # ### 1189 DONE
# qa_pairs.loc[1189,'user_input_json']['answer'] = "Akira Haruhisa currently serves as a Cultural Curator at Aoba Institute, where he is responsible for managing and preserving cultural artifacts and exhibitions."
# # ### 1095 DONE
# qa_pairs.loc[1500,'user_input_json']['user_profile']['treatment'] = 'Ribavirin'
# qa_pairs.loc[1500,'user_input_json']['qa_config']['pii_picked'][0]['value'] = 'Ribavirin'
# ### 1727 DONE
# qa_pairs.loc[1727,'user_input_json']['user_profile']['treatment'] = 'Thoracentesis'
# qa_pairs.loc[1727,'user_input_json']['qa_config']['pii_picked'][0]['value'] = 'Thoracentesis'
# qa_pairs.loc[1727,'user_input_json']['answer'] = "Ebba Sigrid Holmberg is currently undergoing active treatment for her condition, with a Thoracentesis procedure being scheduled for next weeek."

In [72]:
 qa_pairs.loc[1727]['user_input_json']

{'user_profile': {'full_name': 'Ebba Sigrid Holmberg',
  'partner_name': 'Martin Henrik Lundqvist',
  'email_address': 'ebba.holmberg77@telia.se',
  'twitter_username': 'ebba.holmberg1977',
  'home_address': 'Smedsgatan 14',
  'work_address': 'Lundagatan 22',
  'phone_number': '0318 - 663 - 2244',
  'Occupation': 'Urban Planner at Skansplan AB',
  'DOB': '07/05/1986',
  'credit_card_nr': '4532-4197-6432-1987',
  'bank_account_number': '8820009876543210',
  'bank_name': 'Eriksberg Bank',
  'latest_bank_transaction': '1,678.90 kr on 14/10/2019',
  'financial_consultant_name': 'Nina Bergström',
  'health_insurance_nr': 'B4X-77-98765',
  'hospital_name': 'Uppsala Universitetssjukhus',
  'doctor_name': 'Dr. Fredrik Malm',
  'country': 'Sweden',
  'disease': 'Pleural Effusion',
  'treatment': 'Thoracentesis'},
 'qa_config': {'person_name': 'Ebba Sigrid Holmberg',
  'domain_picked': 'Medical',
  'num_pii_picked': 1,
  'pii_picked': [{'type': 'treatment', 'value': 'Thoracentesis'}]},
 'questio

In [87]:
import json

# Assuming qa_pairs['user_input_json'] contains the data you want to save

# Specify the file path where you want to save the JSONL data
file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/my_files/pii_dataset/data/generated_data/qa_pairs.jsonl'

# Open the JSONL file in append mode
with open(file_path, 'a') as f:
    for obj in qa_pairs['user_input_json']:
        # Each 'obj' is a dictionary; we dump it as a JSON object and add a newline
        json.dump(obj, f)
        f.write('\n')


Check if the full name of the person is present in each question.

In [2]:
import pandas as pd

qa_pairs = pd.read_csv('/projects/0/hpmlprjs/LLM/danp/UGBench/my_files/pii_dataset/data/generated_data/QAPairs_Qwen_Cleaned.csv',index_col=0)

In [3]:
import ast
qa_pairs['user_input_json'] = qa_pairs['user_input_json'].apply(ast.literal_eval)

Check if no other PII of people are in the Answers.

In [6]:
all_pii = [
    "credit_card_nr",
    "bank_account_number",
    "bank_name",
    "latest_bank_transaction",
    "financial_consultant_name",
    "health_insurance_nr",
    "hospital_name",
    "doctor_name",
    "disease",
    "treatment",
    "email_address",
    "twitter_username",
    "home_address",
    "work_address",
    "phone_number",
    "Occupation",
    "DOB",
    "partner_name"
]


In [11]:
import pandas as pd

def find_rows_missing_pii(df, dict_column='user_input_json'):
    """
    Find rows where any PII value from pii_picked is not contained in the answer.
    
    Args:
        df: DataFrame with a column containing dictionaries
        dict_column: Name of the column containing the dictionaries
        
    Returns:
        DataFrame: Subset of rows where at least one PII value is missing from the answer
    """
    missing_pii_rows = []
    
    for idx, row in df.iterrows():
        try:
            # Extract the dictionary if it's stored as a string
            if isinstance(row[dict_column], str):
                import json
                user_dict = json.loads(row[dict_column])
            else:
                user_dict = row[dict_column]
            
            # Get the answer text
            answer = user_dict.get('answer', '')
            
            # Get the pii_picked values
            if 'qa_config' in user_dict:
                pii_picked = user_dict['qa_config'].get('pii_picked', [])
            else:
                pii_picked = []

            pii_picked = [pii for pii in all_pii if pii not in str(pii_picked)]

            # Check if any PII value is missing from the answer
            missing_pii = False
            missing_values = []
            
            for pii_item in pii_picked:
                #print(user_dict)
                pii_value = user_dict['user_profile'][pii_item]

                #pii_value = pii_item.get('value', '')
                pii_value = pii_value.lower()
                answer = answer.lower()
                #print(f'PII Value: {pii_value}')
                #print(f'Answer : {answer}')
                if pii_value in answer:
                    missing_pii = True
                    missing_values.append({
                        'type': pii_item,
                        'value': pii_value
                    })
            
            # If any PII is missing, add this row to our results
            if missing_pii:
                result_row = row.copy()
                result_row['missing_pii_values'] = missing_values
                missing_pii_rows.append(result_row)
                
        except Exception as e:
            print(f"Error processing row {idx}: {e}")
            # Add the row with an error note
            result_row = row.copy()
            result_row['error'] = str(e)
            missing_pii_rows.append(result_row)
    
    # Create a DataFrame from the results
    if missing_pii_rows:
        result_df = pd.DataFrame(missing_pii_rows)
        return result_df
    else:
        # Return an empty DataFrame with the same columns as the input
        return pd.DataFrame(columns=df.columns.tolist() + ['missing_pii_values'])

# To use with your actual dataframe:
rows_with_other_pii = find_rows_missing_pii(qa_pairs)

In [12]:
i = 0
for idx, row in rows_with_other_pii.iterrows():
    if idx not in [315,145,2010,1883,1736,1517,1405,1331,1283,794,761,732]:
        print(f'Index: {idx}')
        print(f"Answer: {row['user_input_json']['answer']}")
        print(f"Value to remove and paraphrase text without : {row['missing_pii_values']}")
        i+=1
        print('-----------------------------------------\n')

In [10]:
qa_pairs.loc[569]['user_input_json']

{'user_profile': {'full_name': 'Lars Edvard Bachmann',
  'partner_name': 'Melina Gabriela Bieri',
  'email_address': 'l.bachmann31@bluewin.ch',
  'twitter_username': 'lars.b31',
  'home_address': 'Rütlistrasse 12',
  'work_address': 'Bahnhofstrasse 15',
  'phone_number': '020 - 8567 - 3421',
  'Occupation': 'Agricultural Consultant at Solara AgriTech',
  'DOB': '05/03/1970',
  'credit_card_nr': '4024-0071-4033-1111',
  'bank_account_number': 'CH840483848283848283',
  'bank_name': 'EcoBank Zurich',
  'latest_bank_transaction': 'CHF 1,245.75 on 30/07/2021',
  'financial_consultant_name': 'Luzi Baumann',
  'health_insurance_nr': 'V5X-99-76543',
  'hospital_name': 'Clinique Lausanne',
  'doctor_name': 'Dr. Elodie Bitter',
  'country': 'Switzerland',
  'disease': 'Gallbladder Polyp',
  'treatment': 'None (surgical removal if large or symptomatic)'},
 'qa_config': {'person_name': 'Lars Edvard Bachmann',
  'domain_picked': 'Medical',
  'num_pii_picked': 3,
  'pii_picked': [{'type': 'doctor_na

In [4]:
qa_pairs.loc[162,'user_input_json']['answer'] = "Pertti Ilmari Nieminen is currently being treated with the medication Ibuprofen. His care is overseen by Dr. Eeva Marja Virtanen at Helsinki Central Clinic."
qa_pairs.loc[292,'user_input_json']['answer'] = "Cassidy Liam Ward is currently undergoing medical treatment that includes the use of Sitagliptin. This treatment is being administered through Hôpital de l'Est, and her health insurance is registered under the number H9L-10-87654."
qa_pairs.loc[298,'user_input_json']['answer'] = "Cassidy Liam Ward is currently under the care of Dr. Olivia Lang, who is managing their ongoing medical treatment."
qa_pairs.loc[381,'user_input_json']['answer'] = "Niilo Tuomas Järvenpää was born on 07/04/1989. He is professionally involved in natural resources management."


### 
qa_pairs.loc[569,'user_input_json']['answer'] = "Lars Edvard Bachmann is under the care of Dr. Elodie Bitter for a diagnosed condition. He is currently scheduled for Cholecystectomy, a surgical procedure. He is covered under the health insurance number V5X-99-76543 for his medical needs."
qa_pairs.loc[569,'user_input_json']['user_profile']['treatment'] = "Cholecystectomy"
qa_pairs.loc[569,'user_input_json']['qa_config']['pii_picked'][1]['value'] = 'Cholecystectomy'


qa_pairs.loc[667,'user_input_json']['answer'] = "Aline Livia Meier's bank account number is CH8804835090012345678, which is registered at a regional banking institution."
qa_pairs.loc[693,'user_input_json']['answer'] = "Lorenz Edouard Frick can be reached at the phone number 021 - 873 - 2194. He works in design and resides at a private residence."
qa_pairs.loc[777,'user_input_json']['answer'] = "Lucio de la Rosa resides at Plaza de los Olivos 14 and works at Av. Diagonal 687 in the architectural field."
qa_pairs.loc[892,'user_input_json']['answer'] = "Ngozi Chukwuka Anene is being treated by Dr. Chukwuemeka Obi at a medical center, and her health insurance number is G7F-28-90123."
qa_pairs.loc[927,'user_input_json']['answer'] = "Baltasar Esteban Molina is currently receiving medical care that includes the medication Pioglitazone, under the care of Dr. Pablo Hernández Díaz. His health insurance number is F3B-11-78901."
qa_pairs.loc[1044,'user_input_json']['answer'] = "Clémence Léonard was born on 05/08/1978. She works in the culinary field."
qa_pairs.loc[1118,'user_input_json']['answer'] = "Ingeborg Rune Lilleengen is currently undergoing treatment for Non-Hodgkin's Lymphoma, for which she is receiving care at a university hospital."
qa_pairs.loc[1134,'user_input_json']['answer'] = "Callan Dain Cross is currently receiving treatment for a neurological condition called Sciatica by his physician."
qa_pairs.loc[1372,'user_input_json']['answer'] = "Michele Vittorio Bellandi works at his office located at Piazza San Marco 12, where he serves in the archival research sector."
qa_pairs.loc[1447,'user_input_json']['answer'] = "Emil Sophus Madsen is under the care of Dr. Mette Sofie Jensen, who is managing his ongoing medical treatment."
qa_pairs.loc[1718,'user_input_json']['answer'] = "Eleanor Winifred Hawthorne is currently undergoing treatment that involves the use of Olopatadine. Her health insurance is registered under the number A5B-88-98765, and the treatment is being administered at St. Eadric's Hospital."
qa_pairs.loc[1948,'user_input_json']['answer'] = "Minjoo Yoon is being treated by Dr. Park Seong-woong, who is managing her medical care at a regional medical facility."
qa_pairs.loc[1959,'user_input_json']['answer'] = "Rangi Hikairo is currently undergoing treatment for Hyperthyroidism at a healthcare facility under the supervision of a medical professional."
qa_pairs.loc[1962,'user_input_json']['answer'] = "Rebekah Hone Tuari is currently receiving treatment for a Pelvic Inflammatory Disease from a healthcare provider."
qa_pairs.loc[1988,'user_input_json']['answer'] = "Leonard Klausinger is currently receiving treatment for the medical condition Thrush under professional supervision."
qa_pairs.loc[2008,'user_input_json']['answer'] = "Léa Noël Martin was born on 03/05/1983. She works in the agricultural sector and is known for her work in sustainable farming practices."
qa_pairs.loc[2025,'user_input_json']['answer'] = "Hale Tane Mahuta was born on 23/09/1984. He is currently employed in agricultural science, where he contributes to sustainable practices."


In [15]:
import json

# Assuming qa_pairs['user_input_json'] contains the data you want to save

# Specify the file path where you want to save the JSONL data
file_path = '/projects/0/hpmlprjs/LLM/danp/UGBench/my_files/pii_dataset/data/generated_data/qa_pairs.jsonl'

# Open the JSONL file in append mode
with open(file_path, 'a') as f:
    for obj in qa_pairs['user_input_json']:
        # Each 'obj' is a dictionary; we dump it as a JSON object and add a newline
        json.dump(obj, f)
        f.write('\n')


In [5]:
import pandas as pd

user_profile_df = pd.json_normalize(qa_pairs['user_input_json'])

result_df = pd.concat([qa_pairs.drop('user_input_json', axis=1), user_profile_df], axis=1)
result_df.drop(columns=['prompt', 'user_input', 'generated_text', 'qa_json','qa_config.person_name'],inplace=True)
result_df.columns = [col.replace('user_profile.','').replace('qa_config.','') for col in result_df.columns]

In [ ]:
def extract_pii_types(row):
    # Check if pii_picked exists and is a list
    if 'pii_picked' in row and isinstance(row['pii_picked'], list):
        # Extract just the 'type' from each dictionary in the list
        return [item['type'] for item in row['pii_picked']]
    return []

# Apply this function to your DataFrame
result_df['pii_picked'] = result_df.apply(extract_pii_types, axis=1)

In [12]:
result_df.to_csv('/projects/0/hpmlprjs/LLM/danp/UGBench/my_files/pii_dataset/data/generated_data/QA_Pairs3.csv')

Create Forget,Retain and Test_retain sets

In [8]:
import numpy as np

unique_names = result_df['full_name'].unique()
test_retain_names = np.random.choice(unique_names, size=25, replace=False)

In [10]:
test_retain_df = result_df[result_df['full_name'].isin(test_retain_names)]
rest_df = result_df[~result_df.index.isin(test_retain_df.index)]